# Python Lab: Object-Oriented Programming (OOP) — Learn by Following Along

## What you will learn today
By the end of this notebook, you should be able to:
- Explain what **OOP** is and why we use it
- Define and use **classes** and **objects**
- Explain and use **attributes** and **methods**
- Use `__init__` to initialize objects
- Explain **encapsulation** (controlling access to data) and use a simple Python pattern for it
- Explain **inheritance** (specializing a class) and use it
- Explain **polymorphism** (same interface, different behavior) and see it in action

## How to use this notebook
- Run cells from top to bottom.
- Read the short explanations.
- In the **Try it** cells, change values and re-run.

## Scenario
We will build a tiny "Reading List" system with books. This is small enough to understand, but realistic enough to show why OOP is useful.

## 1) Why OOP?

When programs grow, you need a way to organize code.

**Object-Oriented Programming (OOP)** is one approach where we bundle:
- **Data** (attributes) and
- **Behavior** (methods)

into a single unit called an **object**.

A **class** is a blueprint for building objects.
An **object** is a specific thing created from a class.

Next: we’ll make a class and create objects from it.

## 2) Your first class

### Key vocabulary
- **Class**: a blueprint
- **Object (instance)**: a thing created from the blueprint
- **Method**: a function inside a class
- **`self`**: the current object

Run this example.

In [ ]:
class Greeter:
    # This is a method. Notice the first parameter is `self`.
    def say_hello(self, name: str) -> str:
        return f"Hello, {name}!"


# Create an object (instance) of the class
g = Greeter()

# Call a method on that object
print(g.say_hello("Taylor"))
print(g.say_hello("Jordan"))

### Try it
Change the name below and re-run.

In [ ]:
print(g.say_hello("Your Name"))

## 3) Attributes and `__init__`

Now we’ll build a class that stores information.

### New concept: attributes
**Attributes** are variables attached to an object.

### New concept: `__init__`
`__init__` is a special method that runs automatically when you create a new object.
It is where you set up the object's starting attributes.

We will build a `Book` class with attributes:
- `title`
- `author`
- `pages`
- `status` (unread/reading/finished)

And methods:
- `summary()`
- `set_status()`

Run the next cell.

In [ ]:
class Book:
    """A simple Book class to demonstrate OOP basics."""

    ALLOWED_STATUSES = {"unread", "reading", "finished"}

    def __init__(self, title: str, author: str, pages: int, status: str = "unread"):
        # Validation helps keep objects in a "safe" state.
        if not isinstance(pages, int) or pages <= 0:
            raise ValueError("pages must be a positive integer")
        if status not in self.ALLOWED_STATUSES:
            raise ValueError(f"status must be one of {sorted(self.ALLOWED_STATUSES)}")

        # These are attributes on the object.
        self.title = title
        self.author = author
        self.pages = pages
        self.status = status

    def summary(self) -> str:
        """Return a human-friendly description of this book."""
        return f"{self.title} by {self.author} ({self.pages} pages) - status: {self.status}"

    def set_status(self, new_status: str) -> None:
        """Safely update status."""
        if new_status not in self.ALLOWED_STATUSES:
            raise ValueError(f"new_status must be one of {sorted(self.ALLOWED_STATUSES)}")
        self.status = new_status


b1 = Book("The Hobbit", "J.R.R. Tolkien", 310)
b2 = Book("Dune", "Frank Herbert", 412, "reading")

print(b1.summary())
print(b2.summary())

b1.set_status("finished")
print("After update:")
print(b1.summary())

### What just happened?

- `b1` and `b2` are **objects**.
- Each object has its own **attributes** (title/author/pages/status).
- `summary()` and `set_status()` are **methods**.

### Try it
Create your own book below, print the summary, then change its status.

In [ ]:
my_book = Book("Your Title", "Your Author", 123)
print(my_book.summary())

my_book.set_status("reading")
print(my_book.summary())

## 4) Encapsulation (concept)

**Encapsulation** means: keep an object’s data protected, and interact with it through methods.

In many languages, you can make attributes truly private.
In Python, we mostly use **conventions**:
- A leading underscore (like `_status`) means "treat this as internal".
- We provide methods to safely update it.

Next, we’ll build `SafeBook` where status is stored internally as `_status`.

In [ ]:
class SafeBook:
    """A Book that stores status internally and updates it through a method."""

    ALLOWED_STATUSES = {"unread", "reading", "finished"}

    def __init__(self, title: str, author: str, pages: int, status: str = "unread"):
        if not isinstance(pages, int) or pages <= 0:
            raise ValueError("pages must be a positive integer")
        if status not in self.ALLOWED_STATUSES:
            raise ValueError(f"status must be one of {sorted(self.ALLOWED_STATUSES)}")

        self.title = title
        self.author = author
        self.pages = pages

        # Internal attribute (convention)
        self._status = status

    def get_status(self) -> str:
        return self._status

    def set_status(self, new_status: str) -> None:
        if new_status not in self.ALLOWED_STATUSES:
            raise ValueError(f"new_status must be one of {sorted(self.ALLOWED_STATUSES)}")
        self._status = new_status

    def summary(self) -> str:
        return f"{self.title} by {self.author} ({self.pages} pages) - status: {self.get_status()}"


sb = SafeBook("1984", "George Orwell", 328)
print(sb.summary())

sb.set_status("reading")
print(sb.summary())

print("Reading status via method:", sb.get_status())

### Try it
Below, we show two ways to change status.
- One is recommended (method)
- One is not recommended (directly changing `_status`)

Run the cell and compare.

In [ ]:
# Not recommended (but possible): direct access to an internal attribute
sb._status = "finished"
print("Direct change:", sb.summary())

# Recommended: use the method (gives validation)
sb.set_status("unread")
print("Method change:", sb.summary())

## 5) A class that manages many objects: `ReadingList`

OOP is especially useful when you have many related items.

We’ll create a `ReadingList` class that stores **multiple books** and provides behaviors like:
- Add a book
- Find a book by title
- Remove a book
- Count books by status

Run the next cell.

In [ ]:
from typing import List, Optional


class ReadingList:
    def __init__(self, name: str):
        self.name = name
        self.books: List[Book] = []

    def add_book(self, book: Book) -> None:
        if not isinstance(book, Book):
            raise TypeError("add_book expects a Book instance")
        self.books.append(book)

    def find_by_title(self, title: str) -> Optional[Book]:
        target = title.strip().lower()
        for book in self.books:
            if book.title.strip().lower() == target:
                return book
        return None

    def remove_by_title(self, title: str) -> bool:
        target = title.strip().lower()
        for i, book in enumerate(self.books):
            if book.title.strip().lower() == target:
                self.books.pop(i)
                return True
        return False

    def count_by_status(self) -> dict:
        counts = {"unread": 0, "reading": 0, "finished": 0}
        for book in self.books:
            counts[book.status] += 1
        return counts

    def list_summaries(self) -> List[str]:
        return [book.summary() for book in self.books]


rl = ReadingList("Weekend Reading")
rl.add_book(Book("Dune", "Frank Herbert", 412, "unread"))
rl.add_book(Book("The Pragmatic Programmer", "Andrew Hunt", 352, "reading"))
rl.add_book(Book("Pride and Prejudice", "Jane Austen", 279, "finished"))

print("Reading list:", rl.name)
for s in rl.list_summaries():
    print("-", s)

print("Counts:", rl.count_by_status())

found = rl.find_by_title("dune")
print("Found:", found.summary() if found else None)

removed = rl.remove_by_title("DUNE")
print("Removed Dune?", removed)
print("Now counts:", rl.count_by_status())

### Try it
Add one more book and check the updated counts.

In [ ]:
rl.add_book(Book("Atomic Habits", "James Clear", 320, "unread"))
print("Counts after adding Atomic Habits:", rl.count_by_status())

for s in rl.list_summaries():
    print("-", s)

## 6) Inheritance (concept)

**Inheritance** means: create a new class that is a specialized version of another class.

Example:
- A regular `Book`
- An `EBook` is a `Book` **plus** `file_size_mb`
- An `AudioBook` is a `Book` **plus** `duration_min`

We will:
- Reuse `Book` code using `super()`
- Add new attributes
- Override `summary()` to include extra details

In [ ]:
class EBook(Book):
    def __init__(self, title: str, author: str, pages: int, file_size_mb: float, status: str = "unread"):
        # super() calls the parent (Book) constructor
        super().__init__(title, author, pages, status)

        if not isinstance(file_size_mb, (int, float)) or file_size_mb <= 0:
            raise ValueError("file_size_mb must be a positive number")
        self.file_size_mb = float(file_size_mb)

    def summary(self) -> str:
        # Overriding: we replace/extend the parent method
        base = super().summary()
        return f"{base} (ebook, {self.file_size_mb:.1f} MB)"


class AudioBook(Book):
    def __init__(self, title: str, author: str, pages: int, duration_min: int, status: str = "unread"):
        super().__init__(title, author, pages, status)

        if not isinstance(duration_min, int) or duration_min <= 0:
            raise ValueError("duration_min must be a positive integer")
        self.duration_min = duration_min

    def summary(self) -> str:
        base = super().summary()
        return f"{base} (audiobook, {self.duration_min} min)"


eb = EBook("Python Tricks", "Dan Bader", 302, 3.2, "reading")
ab = AudioBook("Becoming", "Michelle Obama", 448, 1140, "unread")

print(eb.summary())
print(ab.summary())

print("Is EBook a Book?", isinstance(eb, Book))
print("Is AudioBook a Book?", isinstance(ab, Book))

### Try it
Create your own `EBook` or `AudioBook` below.

In [ ]:
my_ebook = EBook("Clean Code", "Robert C. Martin", 464, 5.8, "unread")
print(my_ebook.summary())

my_audio = AudioBook("Deep Work", "Cal Newport", 304, 420, "reading")
print(my_audio.summary())

## 7) Polymorphism (concept)

**Polymorphism** means: "many forms." In programming, it often means:

**Different object types can be used through the same interface**.

In plain English:
- If multiple classes all provide a `summary()` method,
- we can call `summary()` without caring whether the object is a `Book`, `EBook`, or `AudioBook`.

### Why this is powerful
It makes code simpler:
- You don’t need a bunch of `if` statements checking types.
- You can "just call the method" and trust each object to do the right thing.

Run the next cell to see polymorphism in action.

In [ ]:
items = [
    Book("The Hobbit", "J.R.R. Tolkien", 310, "finished"),
    EBook("Python Tricks", "Dan Bader", 302, 3.2, "reading"),
    AudioBook("Becoming", "Michelle Obama", 448, 1140, "unread"),
]

# Polymorphism: the loop doesn't care what each item is.
# It only cares that each item has a summary() method.
for item in items:
    print(item.summary())

### Polymorphism vs. Type Checking
Here is what code might look like WITHOUT polymorphism (not recommended):

- Check the type
- Do different logic

We include it below so you can compare.
Notice how it gets messy.

In [ ]:
for item in items:
    if isinstance(item, EBook):
        print("(type-check)", item.summary())
    elif isinstance(item, AudioBook):
        print("(type-check)", item.summary())
    elif isinstance(item, Book):
        print("(type-check)", item.summary())
    else:
        print("Unknown item")

print("\nNotice: this code does the same thing but is more complicated.")

### Try it
Add another item type into the list and re-run the polymorphism loop.

Question:
- Do you need to change the polymorphism loop?
- Do you need to change the type-checking version?

We’ll do this by making one more subclass: `ComicBook`.

In [ ]:
class ComicBook(Book):
    def __init__(self, title: str, author: str, pages: int, illustrator: str, status: str = "unread"):
        super().__init__(title, author, pages, status)
        self.illustrator = illustrator

    def summary(self) -> str:
        base = super().summary()
        return f"{base} (comic, illustrator: {self.illustrator})"


items.append(ComicBook("Spider-Man: Blue", "Jeph Loeb", 144, "Tim Sale", "unread"))

print("Polymorphism loop:")
for item in items:
    print(item.summary())

print("\nType-checking loop (notice it is now incomplete):")
for item in items:
    if isinstance(item, EBook):
        print("(type-check)", item.summary())
    elif isinstance(item, AudioBook):
        print("(type-check)", item.summary())
    elif isinstance(item, Book):
        print("(type-check)", item.summary())
    else:
        print("Unknown item")

## 8) A practical feature: recommending what to read next

We’ll add a method to `ReadingList` called `recommend_next()`.

Rules:
1. If there is any book with status `"reading"`, recommend the first one.
2. Otherwise, recommend the first `"unread"` book.
3. If none exist, return `None`.

Run the next cell.

In [ ]:
def recommend_next(self) -> Optional[Book]:
    for book in self.books:
        if book.status == "reading":
            return book
    for book in self.books:
        if book.status == "unread":
            return book
    return None


# Attach the method to ReadingList
ReadingList.recommend_next = recommend_next

rl2 = ReadingList("Capstone")
rl2.add_book(Book("Book A", "Author A", 100, "unread"))
rl2.add_book(Book("Book B", "Author B", 120, "reading"))
rl2.add_book(Book("Book C", "Author C", 140, "unread"))

rec = rl2.recommend_next()
print("Recommendation:", rec.summary() if rec else None)

rl2.find_by_title("Book B").set_status("finished")
rec2 = rl2.recommend_next()
print("After finishing Book B:", rec2.summary() if rec2 else None)

### Try it
Change the statuses in the cell above and re-run.

Try:
- Make two books `reading`
- Make all books `finished`
- Start with an empty reading list

## 9) Summary of OOP concepts learned

### Class
A blueprint for creating objects.

### Object (instance)
A specific thing created from a class.

### Attributes
Data stored on an object (like `title`, `pages`).

### Methods
Functions that belong to an object (like `summary()`).

### Encapsulation
Controlling how data is accessed/changed (Python uses conventions like `_status`).

### Inheritance
Create specialized classes based on a parent class (`EBook` is a `Book`).

### Polymorphism
Different classes can share the same method name and be used the same way (`summary()` works for all).

Next, run the self-check cell to confirm everything works.

## 10) Self-check (run this)
If this cell prints `All checks passed.`, the notebook is working correctly.

In [ ]:
# Basic Book
b = Book("Test", "Author", 10)
assert b.status == "unread"
b.set_status("reading")
assert b.status == "reading"

# ReadingList
rl_test = ReadingList("Test")
rl_test.add_book(Book("A", "AA", 1, "unread"))
rl_test.add_book(Book("B", "BB", 2, "reading"))
assert rl_test.find_by_title("a").author == "AA"
assert rl_test.remove_by_title("A") is True
assert rl_test.find_by_title("A") is None

# Inheritance
eb = EBook("E", "EE", 10, 1.5)
ab = AudioBook("Au", "AA", 20, 60)
assert isinstance(eb, Book)
assert isinstance(ab, Book)

# Polymorphism (same method name works)
mixed = [Book("X", "Y", 1), eb, ab, ComicBook("C", "D", 10, "I")]
texts = [item.summary() for item in mixed]
assert len(texts) == 4

print("All checks passed.")

## Reflection prompts
Write short answers (in a separate Canvas discussion or in notes):

1. What problem does OOP help solve compared to writing everything as separate functions?
2. What is one example of encapsulation from this notebook?
3. How did inheritance help us reuse code?
4. Explain polymorphism in your own words. Use the `summary()` examples.
5. If we add a new type of book tomorrow, which parts of the code should **not** need to change?